In [14]:
import pandas as pd
import numpy as np
import re
import jieba
import jieba.posseg as psg

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [5]:
data = pd.read_excel("../data/data_test.xlsx")
dic_file = "../data/test_dict.txt"
stop_file = "../data/test_stopwords.txt"

In [6]:
def chinese_word_cut(mytext):
    jieba.load_userdict(dic_file)
    jieba.initialize()
    try:
        stopword_list = open(stop_file, 'r', encoding='utf-8')
    except:
        stopword_list = []
    
    stop_list = []
    flag_list = ['n', 'nz', 'vn']

    for line in stopword_list:
        line = re.sub(u'\n|\\r', '', line)
        stop_list.append(line)
    
        word_list = []
    #jieba分词
    seg_list = psg.cut(mytext)
    for seg_word in seg_list:
        word = re.sub(u'[^\u4e00-\u9fa5]','',seg_word.word)
        #word = seg_word.word  #如果想要分析英语文本，注释这行代码，启动下行代码
        find = 0
        for stop_word in stop_list:
            if stop_word == word or len(word)<2:     #this word is stopword
                    find = 1
                    break
        if find == 0 and seg_word.flag in flag_list:
            word_list.append(word) 
         
    return (" ").join(word_list)

In [7]:
data['content_cutted'] = data.content.apply(chinese_word_cut)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\legion\AppData\Local\Temp\jieba.cache
Loading model cost 0.695 seconds.
Prefix dict has been built successfully.


In [10]:
def print_top_words(model, feature_names, n_top_words):
    tword = []
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        topic_w = " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        tword.append(topic_w)
        print(topic_w)
    return tword

In [11]:
n_features = 1000 #提取1000个特征词语
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(data.content_cutted)

In [12]:
n_topics = 8
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=50,
                                learning_method='batch',
                                learning_offset=50,
#                                 doc_topic_prior=0.1,
#                                 topic_word_prior=0.01,
                               random_state=0)
lda.fit(tf)

LatentDirichletAllocation(learning_offset=50, max_iter=50, n_components=8,
                          random_state=0)

In [15]:
n_top_words = 25
tf_feature_names = tf_vectorizer.get_feature_names_out()
topic_word = print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
专家 网友 老师 压力 走势 分析 黄金 突破 大盘 问题 股票 建议 新浪 整理 成本 调整 机会 坐堂 趋势 后市 股市 行情 市场 价位 阻力
Topic #1:
比赛 电子竞技 游戏 项目 总决赛 玩家 冠军 战队 赛事 奖金 赛区 世界 星际争霸 俱乐部 决赛 网吧 活动 现场 时间 职业 大赛 全国 国际 组委会 大奖赛
Topic #2:
投资 经济 市场 公司 政府 发展 企业 政策 问题 网站 计划 银行 手机 行业 服务 国家 互联网 资金 基金 业务 管理 影响 记者 文章 信息
Topic #3:
游戏 研究 时间 科学家 人类 人们 世界 玩家 人员 消息 技术 任务 问题 系统 利用 历史 科技 中心 太空 模式 杂志 科学 公司 过程 汽车
Topic #4:
学生 大学 学校 教育 工作 专业 移民 孩子 问题 语言 留学生 情况 能力 国家 家长 计划 国际 课程 论文 英语 生活 费用 学费 培训 家庭
Topic #5:
电影 主持人 票房 影片 观众 演员 作品 朋友 故事 新浪 感觉 合作 娱乐 电影节 现场 角色 记者 时间 主演 媒体 投票 生活 方面 市场 香港电影
Topic #6:
比赛 主队 赔率 球队 主场 客场 火箭 球员 数据 时间 篮板 奇才 公司 本场 足彩 情况 客胜 联赛 状态 问题 体育讯 新浪 助攻 机会 优势
Topic #7:
项目 建筑 生活 发展 地产 新浪 空间 区域 户型 活动 房子 别墅 设计 主持人 文化 产品 国际 市场 中心 艺术 论坛 集团 地方 开发商 客户


In [16]:
topics=lda.transform(tf)

In [17]:
topic = []
for t in topics:
    topic.append("Topic #"+str(list(t).index(np.max(t))))
data['概率最大的主题序号']=topic
data['每个主题对应概率']=list(topics)
data.to_excel("data_topic.xlsx",index=False)